In [1]:
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader

from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_CLASS, N_FFT,
                                           N_MELS, PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH)
from pitch_tracker.utils.dataset import AudioDataset
from pitch_tracker.ml.net import NeuralNetwork, train_model, test_model

e:\Programs\miniconda3\envs\mpt_2022\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\Spaceship420\MY GIT\mono_pitch_tracker\medleydb\medleydb\__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
E:\Spaceship420\MY GIT\mono_pitch_tracker\medleydb\medleydb\__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [6]:
device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"

# device = "cpu"

print(f"Using {device} device")

Using cpu device


In [7]:
DATASET_DIR = '../content/pickled_database/'

dataset_paths = list(files.list_folder_paths_in_dir(DATASET_DIR))
train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
print(f'train_song_set: {len(train_set)}')
print(f'validation_song_set: {len(validation_set)}')
print(f'test_song_set: {len(test_set)}')

train_song_set: 64
validation_song_set: 22
test_song_set: 22


In [66]:
train_dataset = AudioDataset(train_set)
validation_dataset = AudioDataset(validation_set)

# affect GPU dedicated memory
batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = NeuralNetwork().to(device)
loss_fn = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# print(model)

e:\Programs\miniconda3\envs\mpt_2022\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_model(model, train_dataloader,loss_fn, optimizer, device)
    test_model(model, validation_dataloader, loss_fn, device)
print("Done!")

In [64]:
test_model(model, validation_dataloader, loss_fn, device)


Test Error: 
 Accuracy: 3.6%, Avg loss: 0.300722 



In [10]:
X, (y1,y2,y3) = next(iter(train_dataloader))
X = X.to(device)
y3 = y3.to(device)
y_pred = model(X)

In [20]:
y_pred.size()
y3.size()

torch.Size([4, 210, 88])

In [47]:
y_pred.numel()

73920

In [41]:
4*210*88

73920

In [51]:
y_pred.argmax(2).flatten() == y3.argmax(2).flatten()

torch.Size([840])

In [62]:
(y_pred.argmax(2) == y3.argmax(2)).type(torch.float).sum().item()

11.0

In [55]:
pos_neg_matrix = (y_pred.argmax(2) == y3.argmax(2)).flatten()
n_size = pos_neg_matrix.numel()
n_correct = torch.nonzero(pos_neg_matrix).numel()